In [76]:
import pandas as pd
import cv2
import numpy as np

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils


In [77]:
df=pd.read_csv("fer2013.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
emotion    35887 non-null int64
pixels     35887 non-null object
Usage      35887 non-null object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB


In [78]:
print(df["Usage"].value_counts())

Training       28709
PublicTest      3589
PrivateTest     3589
Name: Usage, dtype: int64


In [79]:
X_train,y_train,X_test,y_test=[],[],[],[]

for index,row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
            X_train.append(np.array(val,'float32'))
            y_train.append(row['emotion'])
            
        elif 'PublicTest' in row['Usage']:
            X_test.append(np.array(val,'float32'))
            y_test.append(row['emotion'])
    except:
        print(f"ERROR occured at index:{index} and row:{row}")

In [80]:
print(len(X_train))


28709


In [81]:
X_train[0:2]

[array([ 70.,  80.,  82., ..., 106., 109.,  82.], dtype=float32),
 array([151., 150., 147., ..., 193., 183., 184.], dtype=float32)]

In [82]:
X_train.shape[0]

AttributeError: 'list' object has no attribute 'shape'

In [85]:
X_train=np.array(X_train,'float32')
y_train=np.array(y_train,'float32')
X_test=np.array(X_test,'float32')
y_test=np.array(y_test,'float32')

In [86]:
#Normalizing

X_train-=np.mean(X_train,axis=0)
X_train/=np.std(X_train,axis=0)

X_test-=np.mean(X_test,axis=0)
X_test/=np.std(X_test,axis=0)

In [87]:
num_feats=64
num_labels=7
width,height=48,48

In [88]:
y_train=np_utils.to_categorical(y_train,num_classes=num_labels)
y_test=np_utils.to_categorical(y_test,num_classes=num_labels)


In [89]:
X_train=X_train.reshape(X_train.shape[0],width,height,1)
X_test=X_test.reshape(X_test.shape[0],width,height,1)

In [90]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))

In [91]:
model.compile(loss=categorical_crossentropy,optimizer=Adam(),metrics=['accuracy'])
model.fit(X_train,y_train,batch_size=64,epochs=20,verbose=1,validation_data=(X_test,y_test),shuffle=True)


Train on 28709 samples, validate on 3589 samples
Epoch 1/20
28709/28709 [==============================] - 76s 3ms/step - loss: 1.7296 - accuracy: 0.2899 - val_loss: 1.6338 - val_accuracy: 0.3636
Epoch 2/20
28709/28709 [==============================] - 67s 2ms/step - loss: 1.5198 - accuracy: 0.4020 - val_loss: 1.3911 - val_accuracy: 0.4597
Epoch 3/20
28709/28709 [==============================] - 65s 2ms/step - loss: 1.3963 - accuracy: 0.4569 - val_loss: 1.3299 - val_accuracy: 0.4845
Epoch 4/20
28709/28709 [==============================] - 67s 2ms/step - loss: 1.3366 - accuracy: 0.4799 - val_loss: 1.2769 - val_accuracy: 0.5091
Epoch 5/20
28709/28709 [==============================] - 66s 2ms/step - loss: 1.2886 - accuracy: 0.5047 - val_loss: 1.2415 - val_accuracy: 0.5222
Epoch 6/20
28709/28709 [==============================] - 68s 2ms/step - loss: 1.2486 - accuracy: 0.5193 - val_loss: 1.2247 - val_accuracy: 0.5255
Epoch 7/20
28709/28709 [==============================] - 66s 2ms/st

In [93]:
#saving model

fer_json=model.to_json()
with open("fer.json","w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")

In [ ]:
from keras.applications.vgg16 import VGG16
model = VGG16(include_top=False)

Using TensorFlow backend.



 3588096/58889256 [>.............................] - ETA: 47:38

In [1]:

import os
import cv2
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image

#load model
model = model_from_json(open("fer.json", "r").read())
#load weights
model.load_weights('fer.h5')


face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


cap=cv2.VideoCapture(0)

while True:
    ret,test_img=cap.read()# captures frame and returns boolean value and captured image
    if not ret:
        continue
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.2,2)


    for (x,y,w,h) in faces_detected:
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)
        roi_gray=gray_img[y:y+w,x:x+h]#cropping region of interest i.e. face area from  image
        roi_gray=cv2.resize(roi_gray,(48,48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels /= 255

        predictions = model.predict(img_pixels)

        #find max indexed array
        max_index = np.argmax(predictions[0])

        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        predicted_emotion = emotions[max_index]

        cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    resized_img = cv2.resize(test_img, (1000, 700))
    cv2.imshow('Facial emotion analysis ',resized_img)
    if cv2.waitKey(10) == ord('q'):#wait until 'q' key is pressed
        break

cap.release()
cv2.destroyAllWindows




Using TensorFlow backend.


<function destroyAllWindows>